### Inference rate for randomly created images

In [1]:
import torch
from torchvision import transforms, models
import time
import numpy as np
from PIL import Image

def load_model(model_path):
    # Load the pre-trained model
    model = models.efficientnet_b0(weights=None)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(num_ftrs, 5)
    
    # Load the trained weights
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def preprocess_image(image_size=224):
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def generate_random_images(num_images, image_size=224):
    return [Image.fromarray(np.random.randint(0, 256, (image_size, image_size, 3), dtype=np.uint8)) for _ in range(num_images)]

def measure_inference_time(model, images, preprocess, device, num_runs=100):
    model = model.to(device)
    
    # Warm-up run
    for image in images:
        input_tensor = preprocess(image).unsqueeze(0).to(device)
        _ = model(input_tensor)
    
    # Measure inference time
    start_time = time.time()
    for _ in range(num_runs):
        for image in images:
            input_tensor = preprocess(image).unsqueeze(0).to(device)
            _ = model(input_tensor)
    end_time = time.time()
    
    avg_time = (end_time - start_time) / (num_runs * len(images))
    return avg_time

def main():
    model_path = 'transfer_learning_models/efficientnet_terrain_classifier.pth'
    num_images = 10
    num_runs = 100
    
    model = load_model(model_path)
    preprocess = preprocess_image()
    images = generate_random_images(num_images)
    
    # Test on CPU
    cpu_time = measure_inference_time(model, images, preprocess, torch.device('cpu'), num_runs)
    print(f"Average CPU inference time: {cpu_time*1000:.2f} ms")
    
    # Test on GPU if available
    if torch.cuda.is_available():
        gpu_time = measure_inference_time(model, images, preprocess, torch.device('cuda'), num_runs)
        print(f"Average GPU inference time: {gpu_time*1000:.2f} ms")
    else:
        print("CUDA is not available. GPU inference time not measured.")

if __name__ == "__main__":
    main()

C:\Users\agraw\AppData\Local\Temp\ipykernel_24348\718533513.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=to

Average CPU inference time: 64.54 ms
Average GPU inference time: 10.45 ms


### Inference rate for test images

In [1]:
import torch
from torchvision import transforms, models, datasets
import time
import os
from PIL import Image
from torchsummary import summary

def load_model(model_path):
    model = models.efficientnet_b0(weights=None)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(num_ftrs, 5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def preprocess_image(image_size=224):
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def load_test_images(test_dir, num_images=10):
    test_dataset = datasets.ImageFolder(root=test_dir, transform=None)
    images = []
    for i in range(min(num_images, len(test_dataset))):
        img_path, _ = test_dataset.samples[i]
        img = Image.open(img_path).convert('RGB')
        images.append(img)
    return images

def measure_inference_time(model, images, preprocess, device, num_runs=100):
    model = model.to(device)
    
    # Warm-up run
    for image in images:
        input_tensor = preprocess(image).unsqueeze(0).to(device)
        _ = model(input_tensor)
    
    # Measure inference time
    start_time = time.time()
    for _ in range(num_runs):
        for image in images:
            input_tensor = preprocess(image).unsqueeze(0).to(device)
            _ = model(input_tensor)
    end_time = time.time()
    
    avg_time = (end_time - start_time) / (num_runs * len(images))
    return avg_time

def main():
    model_path = 'transfer_learning_models/efficientnet_terrain_classifier.pth'
    test_dir = r"C:\Users\agraw\Downloads\Testing Data-20241003T161706Z-001\Testing Data"  # Update this with the correct path to your test data
    num_images = 10
    num_runs = 100
    
    model = load_model(model_path)
    preprocess = preprocess_image()
    images = load_test_images(test_dir, num_images)
    print(len(images))
    # summary(model, (3, 224, 224))

    # Test on CPU
    cpu_time = measure_inference_time(model, images, preprocess, torch.device('cpu'), num_runs)
    print(f"Average CPU inference time: {cpu_time*1000:.2f} ms")
    
    # Test on GPU if available
    if torch.cuda.is_available():
        gpu_time = measure_inference_time(model, images, preprocess, torch.device('cuda'), num_runs)
        print(f"Average GPU inference time: {gpu_time*1000:.2f} ms")
    else:
        print("CUDA is not available. GPU inference time not measured.")

if __name__ == "__main__":
    main()

C:\Users\agraw\AppData\Local\Temp\ipykernel_13804\1084491709.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=t

10
Average CPU inference time: 89.56 ms
Average GPU inference time: 15.36 ms


### Additional Statistics 

In [3]:
import torch
from torchvision import models
import os
import sys

def load_model(model_path):
    model = models.efficientnet_b0(weights=None)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(num_ftrs, 5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    return model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_model_size(model_path):
    return os.path.getsize(model_path) / (1024 * 1024)  # Size in MB

def get_model_memory_usage(model, input_size=(1, 3, 224, 224)):
    def get_tensor_size(tensor):
        return tensor.element_size() * tensor.nelement()

    total_memory = 0
    for param in model.parameters():
        total_memory += get_tensor_size(param)
    
    # Estimate memory for input, output, and intermediate activations
    x = torch.rand(input_size)
    model(x)  # Forward pass to initialize lazy modules
    for module in model.modules():
        if hasattr(module, 'weight') and hasattr(module.weight, 'data'):
            total_memory += get_tensor_size(module.weight.data)
        if hasattr(module, 'bias') and hasattr(module.bias, 'data'):
            total_memory += get_tensor_size(module.bias.data)
    
    # Add memory for input and estimated output
    total_memory += get_tensor_size(x)
    total_memory += get_tensor_size(x) * 5  # Assuming 5 classes output

    return total_memory / (1024 * 1024)  # Convert to MB

def main():
    model_path = 'transfer_learning_models/efficientnet_terrain_classifier.pth'
    
    # Load the model
    model = load_model(model_path)
    
    # Get model file size
    model_size = get_model_size(model_path)
    
    # Count parameters
    num_params = count_parameters(model)
    
    # Estimate memory usage
    memory_usage = get_model_memory_usage(model)
    
    print(f"Model File Size: {model_size:.2f} MB")
    print(f"Number of Trainable Parameters: {num_params:,}")
    print(f"Estimated Memory Usage: {memory_usage:.2f} MB")

if __name__ == "__main__":
    main()

Model File Size: 15.60 MB
Number of Trainable Parameters: 4,013,953
Estimated Memory Usage: 34.07 MB


C:\Users\agraw\AppData\Local\Temp\ipykernel_24348\4022171685.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=t